<a href="https://colab.research.google.com/github/taisazero/langchain-crashcourse/blob/main/notebooks/2-simple_agent_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Lab Activity 1: Building a Simple Agent


## Introduction
In this lab, we will be building a simple langchain agent that can use a set of pre-defined tools. We will experiment with adding and removing tools and observe model robustness. At the end of this notebook, we will put our agent together in a chainlit web interface.

**Note:** Refer to the [documentation](https://python.langchain.com/docs/get_started/introduction) and the [deeplearning.ai course](https://learn.deeplearning.ai/langchain) as needed. You are also allowed to ask the TA's and the instructor for help during the class. Please do so if you get stuck and aim to finish the lab within the class time.

First, we need to install our dependencies for this lab.

In [ ]:
#TODO: install langchain
!pip install langchain

In [ ]:
# installing other dependencies
!pip install chainlit
!pip install python-dotenv
!pip install tiktoken
!pip install google-search-results

In [ ]:
%%writefile .env
# TODO: create a .env file with the following variables and then load them using dotenv.
# * SERPAPI_API_KEY (Make one by signing up at https://serpapi.com/)
# * OPENAI_API_KEY
# Note: If you are using the Llama2 hosted model, set the OPENAI_API_KEY to the key provided 
# and also set OPENAI_API_BASE to the base url provided.
OPENAI_API_KEY="YOUR_OPENAI_API_KEY"
# OPENAI_API_BASE="YOUR_OPENAI_API_BASE" # Only if using Llama2 hosted model
SERPAPI_API_KEY="YOUR_SERPAPI_API_KEY"

In [1]:
# TODO: load the environment variables from the .env file using the dotenv package


In [ ]:
#TODO: Create a ChatOpenAI LLM with greedy decoding, a model name of gpt-3.5-turbo, and a max_tokens of 500.
from langchain.chat_models import ChatOpenAI


In [ ]:
#TODO: create a memory, load tools, and initialize an agent with the type `CHAT_CONVERSATIONAL_REACT_DESCRIPTION`
# Tip: Set `return_messages` to `True` in the memory so that the agent returns the messages it generates
# Be sure to set the `memory_key` to `'chat_history'`.

from langchain.memory import ConversationBufferMemory


In [ ]:
#TODO: load the following tools ['llm-math', 'terminal', 'python_repl', 'serpapi']
from langchain.agents import load_tools


In [ ]:
from langchain.agents.initialize import initialize_agent
from langchain.agents import AgentType
 #TODO: initialize agent
    # Hint: There is a parameter that allows you to set the maximum number of iterations, 
        # this is useful to limit the number of messages the agent generates and hence $ spent on OAI credits.
    # Hint: CHAT_CONVERSATIONAL_REACT_DESCRIPTION needs the memory to return its messages.
    # Be sure to set `memory` to the memory object you created above.


In [ ]:
#TODO: run the agent with a set of 5 prompts of your choice and note your observations.
# e.g. agent.run("Hello there!")

## Observations and Insights
Write any observations here. 

Food for thought:
* Try using each tool, mention what works and what doesn't. Why do you think that is? What are some of the limitations of this agent? What are some of the limitations of the tools?
* For searching, try something like `who is the current president of the US` or `Are there any middle eastern arabic restaurants around Charlotte?`. For `math` give it a math word problem e.g. `How many apples are there if I have 5 apples and I buy 10 more?`. For `python-repl` try asking it to make a table of temperatures and compute the average.
* Do you notice any errors?

In [ ]:
# TODO: Re-initialize the agent with the type `CHAT_CONVERSATIONAL_REACT_DESCRIPTION` 
# and set `handle_parsing_errors` to `True`


In [ ]:
#TODO: re-run the agent with a prompt that produced an error in the previous run (if any) and note your observations.


## Optional Observations
What do you notice after adding `handle_parsing_errors = True`? What do you think that does under the hood? 

## Putting Everything Together: Hosting the Agent on Chainlit

Now let's put everything together and build our agent in one cell! The cell below writes to a file called `simple_agent_app.py`. This file will be used to run our agent in the chainlit web interface. This cell will contain code that is indepedent from this notebook since it is written to a standalone python file, so we will be repeating some familiar steps, more practice! It is crucial that you run the cell below before you run the next cell. If you run the next cell first, you will get an error.

In [ ]:
%%writefile simple_agent_app.py
from chainlit.config import config
import chainlit as cl
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
from langchain.agents.initialize import initialize_agent
from langchain.agents import load_tools
from chainlit import on_message, on_chat_start
from langchain.chat_models import ChatOpenAI
from langchain.agents import AgentType

#TODO: load environment variables from your .env file


@cl.on_chat_start
async def start():
    # TODO: initialize LLM (we use ChatOpenAI because we'll later define a `chat` agent), 
    # Hint: How can you enable live generation of the agent's messages? Look into the ChatOpenAI class documentation.
    # BEGIN Writing
   
    #TODO: create a memory, load tools, and initialize an agent with the type `CHAT_CONVERSATIONAL_REACT_DESCRIPTION`
    # Tip: Set `return_messages` to `True` in the memory so that the agent returns the messages it generates
    
    #TODO: load the following tools ['llm-math', 'terminal', 'python_repl', 'serpapi']
    
    #TODO: initialize agent
    # Hint: There is a parameter that allows you to set the maximum number of iterations, 
        # this is useful to limit the number of messages the agent generates and hence $ spent on OAI credits.
    # Hint: CHAT_CONVERSATIONAL_REACT_DESCRIPTION needs the memory to return its messages.
    # Hint: How could you enable the agent to adapt to parsing issues it encounters?
   
    # END Writing
    # create agentexecutor
    cl.user_session.set("agent", agent)

@cl.on_message
async def on_message(message):
    agent = cl.user_session.get("agent")
    response = await cl.make_async(agent.run)(
        input=message, callbacks=[cl.LangchainCallbackHandler()]
    )
    await cl.Message(content=response).send()


Now, let's serve the chainlit web interface with our agent. Run the cell below to start the chainlit web interface. You should see a link to the web interface. Click on the link to open the web interface in a new tab.

In [ ]:
# Note if this cell says the port is already in use, you can change the port number to something else e.g. 8001 and so on.
!chainlit run simple_agent_app.py --port 8000

## Colab Setup
If you are using Colab do not run the cell above, instead run this cell below. It will output a link and an ip-address. Use the link to access your chainlit UI application. It will first prompt you to enter the ip-address, enter the ip-address and click on connect. You should now be able to see the chainlit UI.

In [ ]:
!npm install localtunnel
!chainlit run /content/simple_agent_app.py &>/content/logs.txt & npx localtunnel --port 8000 & curl ipv4.icanhazip.com

## Observations and Insights
Write any observations here. Try using the UI and having a proper conversation with your agent and list your observations for strengths and weaknesses. Inspect the prompt by clicking on the bug icon under the expanded `AgentExecutor` -> `LLMChain`.

Food for thought:
* Try using tools that rely on previous tool outputs for instance, `Are there any middle eastern restaurants around Charlotte?` then ask `What are the reviews for the first option?` Does it remember what the first option is?